In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard

In [ ]:
NUM_DATA_DIR = 'data2/data/GOOGL/'

In [ ]:
# divide data in to three groups
num_test = pd.read_csv(NUM_DATA_DIR+'prices_test.csv')
num_val = pd.read_csv(NUM_DATA_DIR+'prices_val.csv')
num_train = pd.read_csv(NUM_DATA_DIR+'prices_train.csv')

In [ ]:
num_df = pd.concat([num_train,num_val,num_test].axis=0)
attribute = 'Open'
plt.plot(num_df[1000:]['Date'],num_df[1000:][attribute])
plt.xticks(rotation=45)
plt.xlabel('Day')
plt.ylabel(attribute)
plt.show()
del num_df

In [ ]:
from data_util import get_xy

In [ ]:
(x_test,y_test) = get_xy(num_test.values)
(x_train,y_train) = get_xy(num_train.values)
(x_val,y_val) = get_xy(num_val.values)

In [ ]:
# import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, TimeDistributed,concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
numerical_timestep = 20 #  correspond to the 'size' of  the window
attribute_num = 5 # Open/High/Low/AdjClose/Volume 

def build_model(code='Default'):
    numerical_input = Input(shape=(numerical_timestep,attribute_num))
    x = GRU(50,return_sequences=True)(numerical_input)
    x = Dropout(0.2)(x)
    x = GRU(50)(x)
    x = Dropout(0.2)(x)
    # x = Dense(10)(x)
    x = Dense(2,activation='softmax')(x)
    model = Model(inputs=numerical_input,outputs=x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
model = build_model()

In [ ]:
print(model.summary())

In [ ]:
model.fit(x=x_train,y=y_train,batch_size=16,epochs=50,verbose=1,validation_data=(x_val,y_val),\
          callbacks=[TensorBoard(log_dir='num_log/')])

In [ ]:
model.evaluate(x=x_test,y=y_test,batch_size=16)